# **Smarter anomaly detection** - Training an anomaly detection model
*Part 2 - Model training*

## Initialization
---
This repository is structured as follow:

```sh
. smarter-anomaly-detection
|
├── data/
|   ├── interim                          # Temporary intermediate data are stored here
|   ├── processed                        # Finalized datasets ready to be moved to Amazon S3
|   └── raw                              # Immutable original data are stored here
|
└── notebooks/
    ├── 1_data_preparation.ipynb
    ├── 2_model_training.ipynb           <<< THIS NOTEBOOK <<<
    └── 3_model_evaluation.ipynb
```

### Notebook configuration update

In [ ]:
!python -m pip install --upgrade pip
!pip install --quiet --upgrade sagemaker tqdm lookoutequipment

### Imports

In [20]:
import synthetic_config as config
import os
import pandas as pd
import sagemaker

from datetime import datetime

# SDK / toolbox for managing Lookout for Equipment API calls:
import lookoutequipment as lookout

In [21]:
PROCESSED_DATA = os.path.join('..', 'data', 'processed')
TRAIN_DATA     = os.path.join(PROCESSED_DATA, 'train-data')

ROLE_ARN        = sagemaker.get_execution_role()
DATASET_NAME    = config.DATASET_NAME
MODEL_NAME      = config.MODEL_NAME
BUCKET          = config.BUCKET
PREFIX_TRAINING = config.PREFIX_TRAINING
PREFIX_LABEL    = config.PREFIX_LABEL

## Data ingestion
---

In [22]:
lookout_dataset = lookout.LookoutEquipmentDataset(
    dataset_name=DATASET_NAME,
    component_root_dir=f's3://{BUCKET}/{PREFIX_TRAINING}',
    access_role_arn=ROLE_ARN
)
lookout_dataset.create()
response = lookout_dataset.ingest_data(BUCKET, PREFIX_TRAINING)

Dataset "synthetic" does not exist, creating it...



We use the following cell to monitor the ingestion process by calling the following method, which encapsulates the [**DescribeDataIngestionJob**](https://docs.aws.amazon.com/lookout-for-equipment/latest/ug/API_DescribeDataIngestionJob.html) API and runs it every 60 seconds:

In [24]:
lookout_dataset.poll_data_ingestion(sleep_time=15)

2022-04-28 11:00:13 | Data ingestion: IN_PROGRESS
2022-04-28 11:00:28 | Data ingestion: IN_PROGRESS
2022-04-28 11:00:43 | Data ingestion: IN_PROGRESS
2022-04-28 11:00:58 | Data ingestion: IN_PROGRESS
2022-04-28 11:01:13 | Data ingestion: IN_PROGRESS
2022-04-28 11:01:28 | Data ingestion: IN_PROGRESS
2022-04-28 11:01:43 | Data ingestion: IN_PROGRESS
2022-04-28 11:01:58 | Data ingestion: IN_PROGRESS
2022-04-28 11:02:13 | Data ingestion: IN_PROGRESS
2022-04-28 11:02:28 | Data ingestion: IN_PROGRESS
2022-04-28 11:02:43 | Data ingestion: IN_PROGRESS
2022-04-28 11:02:58 | Data ingestion: IN_PROGRESS
2022-04-28 11:03:14 | Data ingestion: IN_PROGRESS
2022-04-28 11:03:29 | Data ingestion: SUCCESS


In case any issue arise, you can inspect the API response available as a JSON document:

In [25]:
lookout_dataset.ingestion_job_response

{'JobId': '4878e28684ff2777faf9e388a98b5928',
 'DatasetArn': 'arn:aws:lookoutequipment:eu-west-1:038552646228:dataset/synthetic/16a8b92f-ffda-4031-b5ca-85f1e4aaa6f7',
 'IngestionInputConfiguration': {'S3InputConfiguration': {'Bucket': 'lookout-equipment-poc',
   'Prefix': 'smarter-anomaly-detection-synth/training-data/'}},
 'RoleArn': 'arn:aws:iam::038552646228:role/service-role/AmazonSageMaker-ExecutionRole-20210903T075832',
 'CreatedAt': datetime.datetime(2022, 4, 28, 10, 59, 48, 948000, tzinfo=tzlocal()),
 'Status': 'SUCCESS',
 'ResponseMetadata': {'RequestId': '4b82d48d-7b4d-46a0-8957-7b100911c8a0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4b82d48d-7b4d-46a0-8957-7b100911c8a0',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '1074',
   'date': 'Thu, 28 Apr 2022 11:03:28 GMT'},
  'RetryAttempts': 0}}

## Model training
---

In [26]:
# Configuring time ranges:
training_start   = pd.to_datetime('2021-01-01 00:00:00')
training_end     = pd.to_datetime('2021-05-31 23:55:00')
evaluation_start = pd.to_datetime('2021-06-01 00:00:00')
evaluation_end   = pd.to_datetime('2021-12-31 23:55:00')

print(f'  Training period | from {training_start} to {training_end}')
print(f'Evaluation period | from {evaluation_start} to {evaluation_end}')

  Training period | from 2021-01-01 00:00:00 to 2021-05-31 23:55:00
Evaluation period | from 2021-06-01 00:00:00 to 2021-12-31 23:55:00


In [27]:
lookout_model = lookout.LookoutEquipmentModel(model_name=MODEL_NAME, dataset_name=DATASET_NAME)
lookout_model.set_time_periods(evaluation_start, evaluation_end, training_start, training_end)
lookout_model.set_label_data(bucket=BUCKET, prefix=PREFIX_LABEL, access_role_arn=ROLE_ARN)
lookout_model.train()

{'ModelArn': 'arn:aws:lookoutequipment:eu-west-1:038552646228:model/synthetic-model/aeb0fe98-b68c-40fc-980a-7c4ced39b68d',
 'Status': 'IN_PROGRESS',
 'ResponseMetadata': {'RequestId': '463e8e85-a95f-4c6d-bd6f-0bfa00d7c658',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '463e8e85-a95f-4c6d-bd6f-0bfa00d7c658',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '144',
   'date': 'Thu, 28 Apr 2022 11:04:15 GMT'},
  'RetryAttempts': 0}}

A training is now in progress. Use the following cell to capture the model training progress. **This model should take around 30-45 minutes to be trained:**

In [28]:
lookout_model.poll_model_training(sleep_time=60)

2022-04-28 11:05:19 | Model training: IN_PROGRESS
2022-04-28 11:06:19 | Model training: IN_PROGRESS
2022-04-28 11:07:19 | Model training: IN_PROGRESS
2022-04-28 11:08:19 | Model training: IN_PROGRESS
2022-04-28 11:09:19 | Model training: IN_PROGRESS
2022-04-28 11:10:19 | Model training: IN_PROGRESS
2022-04-28 11:11:20 | Model training: IN_PROGRESS
2022-04-28 11:12:20 | Model training: IN_PROGRESS
2022-04-28 11:13:20 | Model training: IN_PROGRESS
2022-04-28 11:14:20 | Model training: IN_PROGRESS
2022-04-28 11:15:20 | Model training: IN_PROGRESS
2022-04-28 11:16:20 | Model training: IN_PROGRESS
2022-04-28 11:17:20 | Model training: IN_PROGRESS
2022-04-28 11:18:20 | Model training: IN_PROGRESS
2022-04-28 11:19:21 | Model training: SUCCESS


## Conclusion
---
In this notebook, you ingested the data you prepared previously and trained an anomaly detection model with Amazon Lookout for Equipment.

In the next notebook of this series, you will dive into the results of this trained model.